In [20]:
import tensorflow as tf
from tensorflow import keras
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer

df = pd.read_csv("data/data.csv")

categories = list(df['item'])

X, y = [], []
for label in categories:  
    path = os.path.join('data\\images', label)
    images_list = []
    for img in os.listdir(path):
        image = os.path.join(path, img)
        image = cv2.imread(image)
        image_res = cv2.resize(image, (224, 224))
        image_to_add = img_to_array(image_res)
        
        X.append(image_res)
        y.append(label)
        
X = np.array(X, dtype="float")/ 255.0
y = np.array(y)

In [21]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y)

In [22]:
y

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 1, 0, 0],
       [0, 1, 0, ..., 1, 0, 0],
       [0, 1, 0, ..., 1, 0, 0]])

In [23]:
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.1, 
                                                    shuffle= True)

In [33]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.1,
	      height_shift_range=0.1, horizontal_flip=True)

In [50]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D
from tensorflow.keras.layers import Activation, Flatten, Dropout, Dense, MaxPooling2D

model = Sequential()
input_shape = (224, 224, 3)
channel_dim = -1

model.add(Conv2D(32, (3, 3), padding="same", input_shape=input_shape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=channel_dim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=channel_dim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=channel_dim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=channel_dim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=channel_dim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(len(mlb.classes_)))
model.add(Activation("softmax"))

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [ ]:
model.fit_generator(datagen.flow(train_x, train_y),
          epochs=5,
          validation_data=(test_x, test_y),
          verbose=1)

Epoch 1/5
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
77/77 [==============================] - 573s 7s/step - loss: 1.3184 - acc: 0.6532 - val_loss: 1.5803 - val_acc: 0.6692
Epoch 2/5
35/77 [============>.................] - ETA: 4:44 - loss: 1.2130 - acc: 0.6525

In [ ]:
model.save('smVGG_model.h5')